In [33]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import SparkSession

from shared.constants import DatasetPath

In [35]:
DATASET = DatasetPath('DBLP-V1')

In [36]:
spark = (SparkSession.builder
         .appName(f'{DATASET}')
         .config('spark.sql.legacy.timeParserPolicy', 'LEGACY')
         .config("spark.executor.memory", "8g")
         .config("spark.driver.memory", "8g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "16g")
         .getOrCreate())

In [37]:
df = (
    spark.read.text(DATASET.raw_str('outputacm.txt'), wholetext=False, lineSep='\n\n')
        .withColumn('value', F.split(F.col('value'), '\n'))
)

df.head(10)

[Row(value=['629814', '#*Automated Deduction in Geometry: 5th International Workshop, ADG 2004, Gainesville, FL, USA, September 16-18, 2004, Revised Papers (Lecture Notes in Computer ... / Lecture Notes in Artificial Intelligence)', '#@Hoon Hong,Dongming Wang', '#t2006', '#c', '#index0']),
 Row(value=['#*A+ Certification Core Hardware (Text & Lab Manual)', '#@Charles J. Brooks', '#t2003', '#c', '#index1']),
 Row(value=['#*Performance engineering in industry: current practices and adoption challenges', '#@Ahmed E. Hassan,Parminder Flora', '#t2007', '#cProceedings of the 6th international workshop on Software and performance', '#index2', '#!This panel session discusses performance engineering practices in industry. Presentations in the session will explore the use of lightweight techniques and approaches in order to permit the cost effective and rapid adoption of performance modeling research by large industrial software systems.']),
 Row(value=['#*Dude, You Can Do It! How to Build a Swe

In [38]:
schema = T.StructType([
    T.StructField('title', T.StringType(), True),
    T.StructField('authors', T.ArrayType(T.StringType(), False), False),
    T.StructField('year', T.IntegerType(), True),
    T.StructField('venue', T.StringType(), True),
    T.StructField('index', T.IntegerType(), True),
    T.StructField('references', T.ArrayType(T.StringType(), False), False),
    T.StructField('abstract', T.StringType(), True),
])

@F.udf(returnType=schema)
def parse_citation(lines):
    result = {
        'title': None,
        'authors': [],
        'year': None,
        'venue': None,
        'index': None,
        'references': [],
        'abstract': None,
    }
    for line in lines:
        if line.startswith('#*'):
            result['title'] = line[2:].strip()
        elif line.startswith('#@'):
            result['authors'].extend(line[2:].strip().split(','))
        elif line.startswith('#t'):
            result['year'] = int(line[2:].strip())
        elif line.startswith('#c'):
            result['venue'] = line[2:].strip()
        elif line.startswith('#index'):
            result['index'] = int(line[6:].strip())
        elif line.startswith('#%'):
            result['references'].extend(line[2:].strip().split(','))
        elif line.startswith('#!'):
            result['abstract'] = line[2:].strip()
    return result

df_papers = df.select(
    parse_citation(F.col('value')).alias('parsed_citation')
).select('parsed_citation.*').cache()
df_papers.head(5)

[Row(title='Automated Deduction in Geometry: 5th International Workshop, ADG 2004, Gainesville, FL, USA, September 16-18, 2004, Revised Papers (Lecture Notes in Computer ... / Lecture Notes in Artificial Intelligence)', authors=['Hoon Hong', 'Dongming Wang'], year=2006, venue='', index=0, references=[], abstract=None),
 Row(title='A+ Certification Core Hardware (Text & Lab Manual)', authors=['Charles J. Brooks'], year=2003, venue='', index=1, references=[], abstract=None),
 Row(title='Performance engineering in industry: current practices and adoption challenges', authors=['Ahmed E. Hassan', 'Parminder Flora'], year=2007, venue='Proceedings of the 6th international workshop on Software and performance', index=2, references=[], abstract='This panel session discusses performance engineering practices in industry. Presentations in the session will explore the use of lightweight techniques and approaches in order to permit the cost effective and rapid adoption of performance modeling resea

In [39]:
df_nodes_authors = (
    df_papers.select(
        F.explode(F.col('authors')).alias('name'),
    ).withColumn('id', F.col('name'))
     .filter(F.col('id').isNotNull())
        .dropDuplicates(['id'])
)
print(df_nodes_authors.count())
df_nodes_authors.head(5)

595775


[Row(name=' B. West', id=' B. West'),
 Row(name=' Consumer Electronics Society Staff', id=' Consumer Electronics Society Staff'),
 Row(name=' D. Crookes', id=' D. Crookes'),
 Row(name=' H', id=' H'),
 Row(name=' H. Hsu', id=' H. Hsu')]

In [40]:
df_nodes_venues = (
    df_papers.select(
        F.col('venue').alias('id'),
        F.col('venue').alias('name'),
    ).filter("id != ''")
     .filter(F.col('id').isNotNull())
        .dropDuplicates(['id'])
)
print(df_nodes_venues.count())
df_nodes_venues.head(5)

12609


[Row(id='(1987)', name='(1987)'),
 Row(id='(1992&ndash;1993)', name='(1992&ndash;1993)'),
 Row(id='(1993&ndash;1994)', name='(1993&ndash;1994)'),
 Row(id='(1994 Supplement)', name='(1994 Supplement)'),
 Row(id='(Fall 1991)', name='(Fall 1991)')]

In [41]:
df_papers.groupby('year').count().orderBy('year').show()

+----+-----+
|year|count|
+----+-----+
|  -1|    9|
|1900|    1|
|1941|    1|
|1947|    1|
|1949|    1|
|1950|    2|
|1951|   21|
|1952|   35|
|1953|   63|
|1954|    6|
|1955|   14|
|1956|   53|
|1957|    9|
|1958|   45|
|1959|   89|
|1960|  174|
|1961|  343|
|1962|  412|
|1963|  310|
|1964|  359|
+----+-----+
only showing top 20 rows



In [42]:
df_nodes_papers = (
    df_papers.filter('year > 1900').select(
        F.col('index').alias('id'),
        'title',
        'authors',
        'venue',
        'year',
        'abstract',
        F.to_timestamp(F.col('year').cast('string'), 'yyyy').alias('timestamp')
    ) .filter(F.col('id').isNotNull())
        .dropDuplicates(['id'])
)
print(df_nodes_papers.count())
df_nodes_papers.head(5)

629804


[Row(id=31, title='Beginning Ruby on Rails (Wrox Beginning Guides)', authors=['Steve Holzner'], venue='', year=2006, abstract=None, timestamp=datetime.datetime(2006, 1, 1, 0, 0)),
 Row(id=34, title='Hyperstat: Macintosh Hypermedia for Analyzing Data and Learning Statistics', authors=['David M. Lane'], venue='', year=1993, abstract=None, timestamp=datetime.datetime(1993, 1, 1, 0, 0)),
 Row(id=53, title='Class-specific feature polynomial classifier for pattern classification and its application to handwritten numeral recognition', authors=['Cheng-Lin Liu', 'Hiroshi Sako'], venue='Pattern Recognition', year=2006, abstract='The polynomial classifier (PC) that takes the binomial terms of reduced subspace features as inputs has shown superior performance to multilayer neural networks in pattern classification. In this paper, we propose a class-specific feature polynomial classifier (CFPC) that extracts class-specific features from class-specific subspaces, unlike the ordinary PC that uses a 

In [43]:
df_node_ids = (
    df_nodes_authors.select('id')
        .union(df_nodes_venues.select('id'))
        .union(df_nodes_papers.select('id'))
        .distinct()
)

def filter_node_ids(df):
    return df.join(
        df_node_ids,
        F.col('src') == F.col('id'),
        'inner'
    ).drop(
        'id'
    ).join(
        df_node_ids,
        F.col('dst') == F.col('id'),
        'inner'
    ).drop('id')

In [44]:
df_edges_authored = filter_node_ids(
    df_papers.select(
        F.explode(F.col('authors')).alias('src'),
        F.col('index').alias('dst'),
    ).distinct()
)
print(df_edges_authored.count())
df_edges_authored.head(5)

1337700


[Row(src='Arthur Greef', dst=28),
 Row(src='Lars Dragheim Olsen', dst=28),
 Row(src='Michael Fruergaard Pontoppidan', dst=28),
 Row(src='Palle Agermark', dst=28),
 Row(src='Hans J. Skovgaard', dst=28)]

In [45]:
df_edges_published_in = filter_node_ids(
    df_papers.select(
        F.col('index').alias('src'),
        F.col('venue').alias('dst'),
    ).filter("dst != ''").distinct()
)
print(df_edges_published_in.count())
df_edges_published_in.head(5)

531218


[Row(src=164367, dst='(March 1987)'),
 Row(src=517467, dst='(May 1991)'),
 Row(src=528823, dst='(May 1991)'),
 Row(src=515847, dst='(May 1991)'),
 Row(src=93847, dst='3C ON-LINE')]

In [46]:
df_edges_cited = filter_node_ids(
    df_papers.select(
        F.col('index').alias('src'),
        F.explode(F.col('references')).alias('dst'),
    )
        .withColumn('dst', F.col('dst').cast('int'))
        .distinct()

)
print(df_edges_cited.count())
df_edges_cited.head(5)

632751


[Row(src=55537, dst=26),
 Row(src=391844, dst=76),
 Row(src=408689, dst=101),
 Row(src=56911, dst=101),
 Row(src=62796, dst=126)]

In [47]:
df_nodes_authors.write.parquet(DATASET.processed_str('nodes_Author'), mode='overwrite')
df_nodes_venues.write.parquet(DATASET.processed_str('nodes_Venue'), mode='overwrite')
df_nodes_papers.write.parquet(DATASET.processed_str('nodes_Paper'), mode='overwrite')

df_edges_authored.write.parquet(DATASET.processed_str('edges_AUTHORED'), mode='overwrite')
df_edges_published_in.write.parquet(DATASET.processed_str('edges_PUBLISHED_IN'), mode='overwrite')
df_edges_cited.write.parquet(DATASET.processed_str('edges_CITED'), mode='overwrite')

In [48]:
from datasets.build_schema import build_schema

build_schema(
    spark,
    name=str(DATASET),
    nodes=[
        ('Author', DATASET.processed_str('nodes_Author')),
        ('Venue', DATASET.processed_str('nodes_Venue')),
        ('Paper', DATASET.processed_str('nodes_Paper')),
    ],
    edges=[
        ('Authored', 'Author', 'Paper', DATASET.processed_str('edges_AUTHORED')),
        ('PublishedIn', 'Paper', 'Venue', DATASET.processed_str('edges_PUBLISHED_IN')),
        ('Cited', 'Paper', 'Paper', DATASET.processed_str('edges_CITED')),
    ]
)

[2022-01-19 20:31:32,712][/dd_volume/Development/Python/Thesis/code/datasets/datasets/build_schema.py][DEBUG] Merging old schema for DBLP-V1


DatasetSchema(name='DBLP-V1', prefix='DBLP_V1', database='DBLP-V1', description='None', nodes=[NodeSchema(path='data/processed/DBLP-V1/nodes_Author', properties=[Property(name='name', type='string', ignore=False, label=True, timestamp=False), Property(name='id', type='string', ignore=False, label=False, timestamp=False)], label='Author'), NodeSchema(path='data/processed/DBLP-V1/nodes_Venue', properties=[Property(name='id', type='string', ignore=False, label=False, timestamp=False), Property(name='name', type='string', ignore=False, label=True, timestamp=False)], label='Venue'), NodeSchema(path='data/processed/DBLP-V1/nodes_Paper', properties=[Property(name='id', type='int', ignore=False, label=False, timestamp=False), Property(name='title', type='string', ignore=False, label=True, timestamp=False), Property(name='authors', type='string[]', ignore=False, label=False, timestamp=False), Property(name='venue', type='string', ignore=False, label=False, timestamp=False), Property(name='year'